<a href="https://colab.research.google.com/github/murphycollins/murphycollins.github.io/blob/main/warehouse-migration/notebooks/etl_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install required packages
!pip install pandas sqlalchemy matplotlib seaborn

In [ ]:

import sqlite3

# Connect to local SQLite DB (for demo)
conn = sqlite3.connect("warehouse.db")
c = conn.cursor()

# Create tables
c.execute('''
CREATE TABLE IF NOT EXISTS dim_customer (
    customer_id TEXT PRIMARY KEY,
    name TEXT,
    region TEXT
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS dim_date (
    date_id TEXT PRIMARY KEY,
    date TEXT,
    month INT,
    year INT
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS fact_sales (
    sale_id TEXT PRIMARY KEY,
    date_id TEXT,
    customer_id TEXT,
    store_id TEXT,
    amount REAL
)
''')

conn.commit()
print("✅ Tables created successfully")

In [ ]:

import pandas as pd

# Sample data for customers
customers = pd.DataFrame({
    "customer_id": ["C001","C002","C003"],
    "name": ["Alice","Bob","Charlie"],
    "region": ["North","South","West"]
})

# Load customers into the dim_customer table
customers.to_sql("dim_customer", conn, if_exists="append", index=False)

# Check load
pd.read_sql("SELECT * FROM dim_customer", conn)

In [ ]:

# Insert a sales record
c.execute("INSERT INTO fact_sales VALUES ('S001','D001','C001','Store_1',250.0)")
conn.commit()

pd.read_sql("SELECT * FROM fact_sales", conn)

In [ ]:

query = """
SELECT f.sale_id, d.date, c.name, c.region, f.amount
FROM fact_sales f
LEFT JOIN dim_customer c ON f.customer_id = c.customer_id
LEFT JOIN dim_date d ON f.date_id = d.date_id
"""
pd.read_sql(query, conn)